# Описание проекта:

**Мотивация:** Среди моих знакомых и родных в последнее время часто идут разговоры о недвижимости в москве,  у кого-то она есть, а кто-то планирует переезжать да и я сам хочу пресмотреть что-нибудь в МО. Поэтому решил разобраться в этой теме и используя отрытые данные построить модель оценки стоимости домов и таунхаусов (позже возможно добавлю квартиры).

**Цель:** Создать модель машинного обучения для предсказания стоимости недвижимости (дома и таунхаусы) в Москве и подмосковье с использованием данных, полученных с помощью библиотеки Cianparser.

**Целевая аудитория:**

* Жители Москвы и московской области, владеющие недвижимостью и желающие оценить её стоимость.
* Люди, планирующие переезд в Раменское или Жуковский и интересующиеся ценами на недвижимость.

**Инструменты:**

* Python
* Библиотека Cianparser для сбора данных
* Библиотеки машинного обучения (Scikit-learn)
* Streamlit для создания интерактивного приложения

**Ожидаемые результаты:**

* Модель машинного обучения с MAPE <= 0.2, способная точно предсказывать стоимость недвижимости.
* Интерактивное приложение Streamlit, позволяющее пользователям вводить характеристики недвижимости и получать прогноз цены.

## Парсинг данных

In [114]:
import time
from tqdm import tqdm
import chime
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from bs4 import BeautifulSoup
import cianparser
import requests
import re

chime.theme('mario')
%load_ext chime

The chime extension is already loaded. To reload it, use:
  %reload_ext chime



Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



In [11]:
locs = pd.DataFrame(cianparser.list_locations()) 
locs.columns = ['Город','Индекс']

In [12]:
locs[locs['Город'] == 'Красногорск']

,Город,Индекс
126,Красногорск,175071


In [14]:
locations = ['Жуковский','Раменское','Одинцово','Домодедово',
            'Реутов','Железнодорожный','Долгопрудный','Москва',
            'Красногорск','Подольск','Балашиха']

In [64]:
def parce_suburban(city, suburban_types = ["house",'townhouse'], pages_ranges = [[1,1],[1,1]],
                   parsed_pages=0,return_counter=False, old_df=pd.read_csv('suburban.csv')):
    
    parser = cianparser.CianParser(location=city)
    df = pd.DataFrame()
    data = []

    for suburban_type,pages_range in zip(suburban_types,pages_ranges):
        for page in range(pages_range[0], pages_range[1]+1):
            
            print(f'-----------------------------')
            print(f'Локация: {location}')
            print(f'Вид недвижимости: {suburban_type}')
            print()
            
            # Проверка есть-ли новые ссылки
            fast_suburban_sale = pd.DataFrame(parser.get_suburban( 
                                                deal_type="sale",
                                                suburban_type = suburban_type,
                                                additional_settings={"start_page":page,
                                                                    "end_page":page}
                                                ))
            if fast_suburban_sale.shape[0] > 0:
            
                ids = [x not in old_df['url'].to_list() for x in fast_suburban_sale['url'].to_list()]
                new_df = fast_suburban_sale[ids]
                new_size = new_df.shape[0]

                # Если есть, то скачивается страница с новыми данными
                if new_size > 0:
                    print(f'-----------------------------')
                    print(f'Найдено новых обьявлений: {new_size}')
                    print(f'-----------------------------')
                    if parsed_pages >= 2:
                        num = np.random.randint(100,120)
                        print(f'parsed pages: {parsed_pages}')
                        print(f'timeout: {num}')
                        time.sleep(num)
                        
                    suburban_sale = pd.DataFrame(parser.get_suburban( 
                                                        deal_type="sale",
                                                        suburban_type = suburban_type,
                                                        with_extra_data=True,
                                                        additional_settings={"start_page":page,
                                                                            "end_page":page}
                                                        ))
                    ids = [x not in old_df['url'].to_list() for x in suburban_sale['url'].to_list()]
                    good_sales = suburban_sale[ids]
                    
                    data.append(good_sales)
                    parsed_pages +=1
                else:
                    print(f'-----------------------------')
                    print(f'Новых обьявлений не найдено')
                    print(f'-----------------------------')
                    pass
            else:
                print(f'-----------------------------')
                print(f'Новых обьявлений не найдено')
                print(f'-----------------------------')
                pass

    # Список словарей из полученных данных преобразется в датафрейм

    if len(data) > 0:
        df = pd.concat(data).drop_duplicates().reset_index(drop=True)
    
    if return_counter:
        return df,parsed_pages
    else:
        return df
    
def create_address(row, reverse_order=False, drop_district=False):
    """
    Формирует полный адрес объекта недвижимости из отдельных столбцов DataFrame.

    Args:
        row (pd.Series): Строка DataFrame, содержащая информацию об адресе.
        reverse_order (bool): Если True, формирует адрес в обратном порядке.

    Returns:
        str: Полный адрес объекта недвижимости.
    """
    if reverse_order:
        if drop_district:
            address = f"{row['location']}, {row['street']}, {row['house_number']}" 
        else:
            address = f"{row['location']}, {row['district']}, {row['street']}, {row['house_number']}" 
    else:
        if drop_district:
            address = f"{row['street']}, {row['house_number']}, {row['location']}" 
        else:
            address = f"{row['street']}, {row['house_number']}, {row['district']}, {row['location']}"

    address = address.strip().replace(", , ,", ",").replace(", ,", ",")
    if address.startswith(","):
        address = address[1:]
    return address

def geocode_address(row):
    """
    Геокодирует адрес с помощью OpenStreetMap API.

    Args:
        row (pd.Series): Строка DataFrame, содержащая информацию об адресе.

    Returns:
        tuple: Координаты (latitude, longitude) объекта, если геокодирование успешно, иначе None.
    """
    address = create_address(row.fillna(''))
    try:
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude)
    except:
        try:
            location = geolocator.geocode(address.replace('-я',''))
            return (location.latitude, location.longitude)
        except:
            reversed_address = create_address(row.fillna(''), reverse_order=True)
            try:
                location = geolocator.geocode(reversed_address)
                return (location.latitude, location.longitude)
            except:
                try:
                    reversed_without_district_address = create_address(row.fillna(''), reverse_order=True, drop_district=True)
                    location = geolocator.geocode(reversed_without_district_address)
                    return (location.latitude, location.longitude)
                except:
                    try:
                        without_district_address = create_address(row.fillna(''), drop_district=True)
                        location = geolocator.geocode(without_district_address)
                        return (location.latitude, location.longitude)
                    except:
                        try:
                            location = geolocator.geocode(without_district_address.replace('-я',''))
                            return (location.latitude, location.longitude)
                        except:
                            try:
                                only_city = row['location']
                                location = geolocator.geocode(only_city)
                                return (location.latitude, location.longitude)
                            except:
                                return None

In [65]:
%%time
%%chime
dfs = []
old_df = pd.read_csv('suburban.csv')
counter = 0

for location in locations:
    
    df,counter = parce_suburban(location, parsed_pages = counter, return_counter = True,
                                old_df=old_df)
    if not df.empty:
        dfs.append(df)

-----------------------------
Локация: Жуковский
Вид недвижимости: house


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 34 404 103 rub

The collection of information from the pages with list of offers is completed
Total number of parsed offers: 28. 
-----------------------------
Новых обьявлений не найдено
-----------------------------
-----------------------------
Локация: Жуковский
Вид недвижимости: townhouse


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750

In [66]:
df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
df['date'] = dt.date.today()
df

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,ID 461204,homeowner,https://odintsovo.cian.ru/sale/suburban/300340...,Жуковский,sale,suburban,townhouse,-1,0,35000000,...,-1,-1,150 м²,2,+79850012066,,,Вишневая улица,,2024-04-25
1,Кредит Центр,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29951...,Раменское,sale,suburban,house,-1,0,8900000,...,-1,-1,"133,8 м²",2,+79660617340,р-н Центральный,м. Раменское,,,2024-04-25
2,BARNES International Realty,real_estate_agent,https://odintsovo.cian.ru/sale/suburban/292072...,Одинцово,sale,suburban,house,-1,0,146354523,...,-1,-1,700 м²,2,+79623650297,,м. Мичуринец,Баковка-Набережная ДНТ,1В,2024-04-25
3,ID 76943077,unknown,https://odintsovo.cian.ru/sale/suburban/278034...,Одинцово,sale,suburban,house,-1,0,43700000,...,-1,-1,-1,2,+79660624795,,м. Одинцово,,,2024-04-25
4,FS PROPERTY,real_estate_agent,https://odintsovo.cian.ru/sale/suburban/219764...,Одинцово,sale,suburban,house,-1,0,513104899,...,-1,-1,-1,-1,+79175790370,,,,,2024-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,ЕГСН Продажа,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30112...,Балашиха,sale,suburban,house,-1,0,25700000,...,-1,-1,168 м²,2,+79645647890,,м. Кучино,Коллективная улица,165Б,2024-04-25
71,Гарант плюс,real_estate_agent,https://balashikha.cian.ru/sale/suburban/28937...,Балашиха,sale,suburban,house,-1,0,23000000,...,-1,-1,286 м²,2,+79647220890,,м. Железнодорожная,улица Лермонтова,30,2024-04-25
72,ОГРК,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30087...,Балашиха,sale,suburban,townhouse,-1,0,9800000,...,-1,-1,220 м²,2,+79672089940,,,,,2024-04-25
73,Сания Галиева,realtor,https://balashikha.cian.ru/sale/suburban/29976...,Балашиха,sale,suburban,townhouse,-1,0,18900000,...,-1,-1,180 м²,-1,+79165625420,,,улица Садовая,42,2024-04-25


In [67]:
%%chime

new_df = df[[x not in old_df['url'].to_list() for x in df['url'].to_list()]]
new_size = new_df.shape[0]
display(new_df)

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,ID 461204,homeowner,https://odintsovo.cian.ru/sale/suburban/300340...,Жуковский,sale,suburban,townhouse,-1,0,35000000,...,-1,-1,150 м²,2,+79850012066,,,Вишневая улица,,2024-04-25
1,Кредит Центр,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29951...,Раменское,sale,suburban,house,-1,0,8900000,...,-1,-1,"133,8 м²",2,+79660617340,р-н Центральный,м. Раменское,,,2024-04-25
2,BARNES International Realty,real_estate_agent,https://odintsovo.cian.ru/sale/suburban/292072...,Одинцово,sale,suburban,house,-1,0,146354523,...,-1,-1,700 м²,2,+79623650297,,м. Мичуринец,Баковка-Набережная ДНТ,1В,2024-04-25
3,ID 76943077,unknown,https://odintsovo.cian.ru/sale/suburban/278034...,Одинцово,sale,suburban,house,-1,0,43700000,...,-1,-1,-1,2,+79660624795,,м. Одинцово,,,2024-04-25
4,FS PROPERTY,real_estate_agent,https://odintsovo.cian.ru/sale/suburban/219764...,Одинцово,sale,suburban,house,-1,0,513104899,...,-1,-1,-1,-1,+79175790370,,,,,2024-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,ЕГСН Продажа,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30112...,Балашиха,sale,suburban,house,-1,0,25700000,...,-1,-1,168 м²,2,+79645647890,,м. Кучино,Коллективная улица,165Б,2024-04-25
71,Гарант плюс,real_estate_agent,https://balashikha.cian.ru/sale/suburban/28937...,Балашиха,sale,suburban,house,-1,0,23000000,...,-1,-1,286 м²,2,+79647220890,,м. Железнодорожная,улица Лермонтова,30,2024-04-25
72,ОГРК,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30087...,Балашиха,sale,suburban,townhouse,-1,0,9800000,...,-1,-1,220 м²,2,+79672089940,,,,,2024-04-25
73,Сания Галиева,realtor,https://balashikha.cian.ru/sale/suburban/29976...,Балашиха,sale,suburban,townhouse,-1,0,18900000,...,-1,-1,180 м²,-1,+79165625420,,,улица Садовая,42,2024-04-25


In [129]:
%%time
%%chime

if new_df.shape[0] > 0:
    geolocator = Nominatim(user_agent="my_app", timeout=60)

    # moscow_center = geolocator.geocode("Москва, центр")
    # moscow_coords = (moscow_center.latitude, moscow_center.longitude)
    new_df['address'] = new_df[geo_cols].fillna('').apply(create_address, axis=1)

    coordinates = []
    for index, row in tqdm(new_df.iterrows(), total=new_df.shape[0]):
        coordinates.append(geocode_address(row))

    new_df['coordinates'] = coordinates

    num_missing_coords = new_df['coordinates'].isna().sum()
    num_geocoded = len(new_df) - num_missing_coords
    print(f"Количество объектов с координатами: {num_geocoded}")
    print(f"Количество объектов без координат: {num_missing_coords}")

    full_df = (pd.concat([old_df,new_df])
            .drop_duplicates(subset=['url','author'],ignore_index=True)
            .reset_index(drop=True))

    full_df.to_csv('suburban.csv',index=False)

    
    print(f'Количество новых обьялений: {new_size}')

else:
    full_df = old_df
    print('Новых обьявлений не обнаруженно')

print(f'Суммарное количество объялений: {full_df.shape[0]}')


Количество новых обьялений: 75
Суммарное количество объялений: 2886
